# Importing necessary libraries and data

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from IPython.display import display
from sklearn.model_selection import train_test_split
import xgboost as xgb
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile

# Provide the path to your zip file and the directory where you want to unzip its contents
zip_file_path = '/content/drive/MyDrive/DatsetForUse.zip'  # Update with your file path
extracted_dir_path = '/content/B_vs_NB/'  # Update with your desired directory

os.makedirs(extracted_dir_path, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

# List the extracted files and directories
extracted_files = os.listdir(extracted_dir_path)
print("Extracted files and directories:", extracted_files)


In [ ]:
# Specify the directory path you want to list files from
folder_path = '/content/B_vs_NB/DatsetForUse/Images_Bleeding'  # Update with your directory path

# Use os.listdir to get a list of all files and directories in the specified folder
files_and_dirs = os.listdir(folder_path)

# Filter the list to get only file paths (excluding directories)
file_paths1 = [os.path.join(folder_path, item) for item in files_and_dirs if os.path.isfile(os.path.join(folder_path, item))]



In [ ]:
# Specify the directory path you want to list files from
folder_path = '/content/B_vs_NB/DatsetForUse/Images_NonBleeding'  # Update with your directory path

# Use os.listdir to get a list of all files and directories in the specified folder
files_and_dirs = os.listdir(folder_path)

# Filter the list to get only file paths (excluding directories)
file_paths2 = [os.path.join(folder_path, item) for item in files_and_dirs if os.path.isfile(os.path.join(folder_path, item))]

In [ ]:
import pandas as pd

# Create a list of file paths and their corresponding classes
file_paths = file_paths1 + file_paths2
classes = ['Bleeding'] * len(file_paths1) + ['NonBleeding'] * len(file_paths2)

# Create a Pandas DataFrame
data = {'filepath': file_paths, 'Class': classes}
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['filepath'], df['Class'], test_size=0.2)

# 1) ResNET152V2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the input image size
target_size = (224, 224)
img_channels = 3  #  RGB images
batch_size = 32


train_df = pd.DataFrame({'filepath': X_train, 'label': y_train})
test_df = pd.DataFrame({'filepath': X_test, 'label': y_test})

# Load the pre-trained ResNet152v2 model without the top classification layer
base_model = ResNet152V2(input_shape=(target_size[0], target_size[1], img_channels),
                   include_top=False,
                   weights='imagenet')

# Freeze the layers of ResNet152v2
base_model.trainable = False

# Create an ImageDataGenerator for preprocessing and data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
)

# Load and preprocess training data
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True  # Shuffle the training data
)

# Load and preprocess validation data
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Do not shuffle the validation data
)

# Create the model by adding additional layers for classification
model = models.Sequential([
    base_model,
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

# Compile the model and specify loss, optimizer, and metrics as needed
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

# Evaluate the model
evaluation = model.evaluate(test_generator)
print("Evaluation Loss:", evaluation[0])
print("Evaluation Accuracy:", evaluation[1])


In [ ]:
model.save('Wce_resnet.h5') #save the model
print(model.summary())

In [ ]:
from sklearn.metrics import classification_report

# Assuming you have already trained your model and obtained predictions
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

true_labels = test_generator.classes

# Get the filenames of test images
filepaths = test_generator.filepaths

# Initialize lists to store misclassified filepaths
misclassified_filepaths = []

# Iterate over predictions and true labels to find misclassifications
for i in range(len(true_labels)):
    if predicted_labels[i] != true_labels[i]:
        misclassified_filepaths.append(filepaths[i])

# Print the misclassified filepaths
print("Misclassified Filepaths:")
for filepath in misclassified_filepaths:
    print(filepath)

# Evaluation
evaluation = model.evaluate(test_generator)
print("Evaluation Loss:", evaluation[0])
print("Evaluation Accuracy:", evaluation[1])

# Classification Report
report = classification_report(true_labels, predicted_labels)
print("Classification Report:\n", report)


# 2) MobileNETv2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the input image size
target_size = (224, 224)
img_channels = 3  # RGB images
batch_size = 32


train_df = pd.DataFrame({'filepath': X_train, 'label': y_train})
test_df = pd.DataFrame({'filepath': X_test, 'label': y_test})

# Load the pre-trained MobileNETv2 model without the top classification layer
base_model = MobileNetV2(input_shape=(target_size[0], target_size[1], img_channels),
                   include_top=False,
                   weights='imagenet')

# Freeze the layers of MobileNETv2
base_model.trainable = False

# Create an ImageDataGenerator for preprocessing and data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
)

# Load and preprocess training data
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True  # Shuffle the training data
)

# Load and preprocess validation data
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Do not shuffle the validation data
)

# Create the model by adding additional layers for classification
model = models.Sequential([
    base_model,
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

# Compile the model and specify loss, optimizer, and metrics as needed
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

# Evaluate the model
evaluation = model.evaluate(test_generator)
print("Evaluation Loss:", evaluation[0])
print("Evaluation Accuracy:", evaluation[1])


In [ ]:
model.save('Wce_mobilenet.h5')
print(model.summary())

In [ ]:
from sklearn.metrics import classification_report
# Generate predictions for the test data
predictions = model.predict(test_generator)

# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = test_generator.classes

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Generate classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

print("Classification Report:\n", report)

# Run Grad-CAM on sample images

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.image as mpimg

# Load the trained model
model = load_model('Wce_resnet.h5')
# Get the name of the last convolutional layer in the model
last_conv_layer_name = model.layers[-5].name
print(last_conv_layer_name)

# Prepare an example image for Grad-CAM
img_path = '/content/B_vs_NB/DatsetForUse/Images_Bleeding/img- (1255).png'  # Replace with the path to your example image
img = mpimg.imread(img_path)
plt.imshow(img)
plt.show()
print(img.shape)


grad_model = tf.keras.models.Model(
    [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
)

target_size = (224, 224)
img = tf.image.resize(img, target_size)
img=np.expand_dims(img,axis=0)
num_zeros = np.count_nonzero(img == 0)
print(num_zeros)

print(model.predict(img))
pred_index=None
with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

# This is the gradient of the output neuron (top predicted or chosen)
# with regard to the output feature map of the last conv layer
grads = tape.gradient(class_channel, last_conv_layer_output)
print(grads.shape)
num_zeros = np.count_nonzero(grads == 0)
print(num_zeros)

# This is a vector where each entry is the mean intensity of the gradient
# over a specific feature map channel
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

# We multiply each channel in the feature map array
# by "how important this channel is" with regard to the top predicted class
# then sum all the channels to obtain the heatmap class activation
last_conv_layer_output = last_conv_layer_output[0]
heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
heatmap = tf.squeeze(heatmap)

# For visualization purpose, we will also normalize the heatmap between 0 & 1
heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
heatmap=heatmap.numpy()

plt.matshow(heatmap)
plt.show()

Superimpose the heat map on original image and save it

In [ ]:
# Load the original image
img = keras.utils.load_img(img_path)
img = keras.utils.img_to_array(img)

# Rescale heatmap to a range 0-255
heatmap = np.uint8(255 * heatmap)

alpha = 0.4

# Use jet colormap to colorize heatmap
jet = mpl.cm.get_cmap("jet")

# Use RGB values of the colormap
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

# Create an image with RGB colorized heatmap
jet_heatmap = keras.utils.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = keras.utils.img_to_array(jet_heatmap)

# Superimpose the heatmap on the original image
superimposed_img = jet_heatmap * alpha + img
superimposed_img = keras.utils.array_to_img(superimposed_img)

# Save the superimposed image
cam_path = '/content/cam.jpg'
superimposed_img.save(cam_path)
superimposed_img.save('/content/cam1255.pdf')

# Open the image
image = Image.open(cam_path)

# Show the plot
plt.imshow(image)
plt.show()
